### SlurmDriver Example

In [26]:
from slurmflow.driver import SlurmDriver
from slurmflow.config import ConfigParser
from os.path import abspath

The `SlurmDriver` class is capable of interfacting with the __S__imple __L__inux __U__tility for __R__esource __M__anagement. `SlurmDriver` can submit jobs and monitor the queue. 

First we will load a configuration for the `SlurmDriver` using the `ConfigParser` (see `config.ipynb`)

In [27]:
cfg = ConfigParser('config/slurm.yml')
cfg.compile()

{'partition': 'standard',
 'time': '01:00:00',
 'nodes': 1,
 'tasks-per-node': 1,
 'output_dir': './logs'}

Note that relative paths are used in the configuration file. SLURM prefers absolute paths, so we will use the `get` and `set` methods of the `ConfigParser` to set `output_dir` to its absolute path.

In [28]:
cfg.set('output_dir', abspath(cfg.get('output_dir')))
cfg.compile()

{'partition': 'standard',
 'time': '01:00:00',
 'nodes': 1,
 'tasks-per-node': 1,
 'output_dir': '/Users/lukasherron/Documents/GitHub/slurm-flow/logs'}

The config is also missing the `job_name` SBATCH directive. We will set it in-memory as well.

In [29]:
cfg.set("job_name", "hello_world")
cfg.compile()

{'partition': 'standard',
 'time': '01:00:00',
 'nodes': 1,
 'tasks-per-node': 1,
 'output_dir': '/Users/lukasherron/Documents/GitHub/slurm-flow/logs',
 'job_name': 'hello_world'}

As you can probably guess, the submitted command will be `python -c print("Hello World!")`

In [30]:
cmd='python -c print("Hello World!")'

In [ ]:
driver = SlurmDriver()
driver.submit_job(cmd, 
                  slurm_args=cfg.compile(),
                  env='', # optionally provide the name of a conda or mamba env (see venv kwarg).
                  modules=[] # optionally provide modules to load
                  track=True # add jobid to the registry
                 ) 